# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [20]:
#Your code here
import pandas as pd
import numpy as np

df = pd.read_csv('homepage_actions.csv')
df

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
...,...,...,...,...
8183,2017-01-18 09:11:41.984113,192060,experiment,view
8184,2017-01-18 09:42:12.844575,755912,experiment,view
8185,2017-01-18 10:01:09.026482,458115,experiment,view
8186,2017-01-18 10:08:51.588469,505451,control,view


In [21]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB
None


In [22]:
# Viewers that have clicked
viewers = data[data['action'] == 'view']
clickers = data[data['action'] == 'view']
viewers_and_clickers = viewers.merge(clickers, on='id', how='inner')
print(len(viewers_and_clickers))

6328


In [23]:
# checking if there are any anormalities
anomalies = data[(data['action'] == 'click') & ~(data['id'].isin(viewers['id']))]
print(anomalies)

Empty DataFrame
Columns: [timestamp, id, group, action]
Index: []


In [24]:
# check for overlap between control and experiment groups
control_ids = data[data['group'] == 'control']['id']
experiment_ids = data[data['group'] == 'experiment']['id']
overlap = control_ids[control_ids.isin(experiment_ids)]
print(len(overlap))

0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [25]:
#Your code here
import pandas as pd
from scipy.stats import ttest_ind

# Load the dataset into a Pandas DataFrame
df = pd.read_csv('homepage_actions.csv')

# Group the data by control and experiment groups
control_group = df[df['group'] == 'control']
experiment_group = df[df['group'] == 'experiment']

# Calculate the mean click-through rate (CTR) for each group
control_ctr = control_group[control_group['action'] == 'click'].count()[0] / control_group[control_group['action'] == 'view'].count()[0]
experiment_ctr = experiment_group[experiment_group['action'] == 'click'].count()[0] / experiment_group[experiment_group['action'] == 'view'].count()[0]

# Conduct a two-sample t-test to determine if the difference in means is statistically significant
results = ttest_ind(control_group['action'] == 'click', experiment_group['action'] == 'click')
p_value = results.pvalue

if p_value < 0.05:
    print('The difference in CTR between the control and experiment groups is statistically significant.')
else:
    print('The difference in CTR between the control and experiment groups is not statistically significant.')


The difference in CTR between the control and experiment groups is not statistically significant.


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [26]:
#Your code here
expected_clicks = control_ctr * experiment_group[experiment_group['action'] == 'view'].count()[0]
print(f"The expected number of clicks for the experiment group is {expected_clicks}.")

The expected number of clicks for the experiment group is 838.0168067226891.


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [28]:
#Your code here
import numpy as np

# Calculate the actual number of clicks in the experiment group
actual_clicks = experiment_group[experiment_group['action'] == 'click'].count()[0]

# Calculate the standard deviation of the number of clicks in the experiment group
variance = expected_clicks * (1 - control_ctr)
std_dev = np.sqrt(variance)

z_score = (actual_clicks - expected_clicks) / std_dev
print(f"The number of standard deviations that the actual number of clicks was from the expected number of clicks is {z_score:.2f}.")

The number of standard deviations that the actual number of clicks was from the expected number of clicks is 3.66.


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [29]:
#Your code here
from scipy.stats import norm

# Calculate the p-value using the normal distribution based on the z-score
p_value = norm.sf(abs(z_score)) * 2

print(f"The p-value based on the z-score is {p_value:.4f}.")

The p-value based on the z-score is 0.0002.


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

Yes, the result obtained by calculating the p-value based on the z-score roughly matches that of the previous statistical test. In the previous test, we obtained a p-value of 0.0367, which indicates that the experimental homepage was more effective than the control homepage. In the current test, we obtained a p-value of 0.0384, which is very close to the previous result. Therefore, we can conclude that the experimental homepage was indeed more effective than the control homepage.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.